This Jupyter notebook is responsible for running the cancer detection algorithm.

# Data preprocessing

In [ ]:
import pandas as pd
import methods
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, f1_score, recall_score, classification_report
from skimage.feature import graycomatrix, graycoprops
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
import math
from skimage import data, feature, color, io
from skimage.draw import disk


%load_ext autoreload
%autoreload 2

#original_folder_path =  './dataverse_files/HAM10000_images_part_1'
#original_folder_path =  './dataverse_files/HAM10000_images_part_1_2'  # folder containing all 10k images
original_folder_path =  './dataverse_files/JS_Selection'
processed_folder_path =  './preprocessed_images'

if not os.path.exists(processed_folder_path):
    os.makedirs(processed_folder_path)

df = pd.read_csv("./dataverse_files/HAM10000_metadata.csv")


df.dx.unique()

cancerous = ["akiec", "bcc", "mel"]
non_cancerous = ["bkl", "df", "nv", "vasc"]
#df["cancer"] = df.dx.isin()
df["cancer"] = False

# Assign True to 'cancer' where 'dx' matches the cancerous list
df.loc[df['dx'].isin(cancerous), 'cancer'] = True
df.loc[(df['dx'].isin(non_cancerous)), 'cancer'] = False


In [ ]:
len(df[df.cancer==True]), len(df[df.cancer==False])

In [ ]:
for image_name in os.listdir(processed_folder_path):
    full_image_path = os.path.join(processed_folder_path, image_name)
    print(methods.lbp_features(full_image_path, radius=1, n_points=8, method='uniform'))

In [ ]:
for image_name in os.listdir(original_folder_path):
    print("Processing", image_name)
    image_path = os.path.join(original_folder_path, image_name)
    
    if image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        #processed_img = load_preprocess_image(image_path)
        processed_img = cv2.imread(image_path)
                                           
        processed_image_path = os.path.join(processed_folder_path, image_name)
        cv2.imwrite(processed_image_path, processed_img)

In [ ]:
image_path = os.path.join(original_folder_path, os.listdir(original_folder_path)[4])
image = cv2.imread(image_path)
# Convert the color from BGR to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert the color from BGR to RGB
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blob = methods.detect_significant_blob(image, plot_image=True, plot_chosen_transformation=False)

#methods.calculate_glcm_features_for_blob(gray_image, blob)
#plt.imshow(image)

In [ ]:
hist = methods.create_histogram(image, color_space="HSV")
plt.plot(hist[0], "r")
plt.plot(hist[1], "g")
plt.plot(hist[2], "b")

In [ ]:
histograms_cancer = []
histograms_benign = []
graycomatrix_features = []

for image_name in tqdm(os.listdir(original_folder_path)):
    image_path = os.path.join(original_folder_path, image_name)

    if image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        cancer = df.loc[df['image_id'] == image_name.split(".")[-2], 'cancer'].values[0]

        # Read the image using OpenCV
        image = cv2.imread(image_path)
        # Convert the color from BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # get the Histogram
        hist = methods.create_histogram(image, color_space="HSV")

        """Texture Extraction"""
        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Normalize pixel values to 0 - 255
        gray_image = np.uint8((gray_image - gray_image.min()) / (gray_image.max() - gray_image.min()) * 255)

        """
        # Compute the Gray-Level Co-Occurrence Matrix (GLCM)
        # distances [1] and angles [0, np.pi/4, np.pi/2, 3*np.pi/4] (0, 45, 90, and 135 degrees)
        glcm = graycomatrix(gray_image, distances=[1, 2, 3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)


        # Feature extraction
        properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
        feature_vector = []
        for prop in properties:
            temp = graycoprops(glcm, prop).flatten()  # Flatten to convert from 2D to 1D
            feature_vector.append(np.mean(temp))  # Taking mean across different angles
        """
        
        blob = methods.detect_significant_blob(image, plot_image=False)
        feature_vector = methods.calculate_glcm_features_for_blob(gray_image, blob)
        graycomatrix_features.append(feature_vector)
           

        if cancer==True:
            histograms_cancer.append(hist)
        elif cancer==False:
            histograms_benign.append(hist)
        else:
            assert("False")


histograms_benign_avg = np.mean(histograms_benign, axis=0)
histograms_cancer_avg = np.mean(histograms_cancer, axis=0)

In [ ]:
np.shape(graycomatrix_features)

In [ ]:
histograms_cancer_ = np.array(histograms_cancer).squeeze()
histograms_benign_ = np.array(histograms_benign).squeeze()

X = np.concatenate((histograms_cancer_, histograms_benign_), axis=0)

Y = np.concatenate((np.ones(histograms_cancer_.shape[0]), -1*np.ones(histograms_benign_.shape[0])), axis=0)

X_reshaped = X.reshape(len(Y), -1) #  flatten the RGB channels
np.shape(X_reshaped)

In [ ]:
#X_reshaped = np.concatenate((X_reshaped, graycomatrix_features), axis=1) # add additional featuers
X_reshaped = graycomatrix_features
np.shape(X_reshaped)

In [ ]:
# Initialize PCA, let's say we want to keep 95% of the variance
pca = PCA(n_components=0.95)

# Fit and transform the data
X_pca = pca.fit_transform(X_reshaped)

# Check the new shape of the data
print(X_pca.shape)


In [ ]:
import matplotlib.pyplot as plt

# Scatter plot of the first two PCA components
# Here, X_pca[:, 0] is the first component, X_pca[:, 1] is the second component
plt.figure(figsize=(10, 7))
plt.scatter(X_pca[Y == -1, 0], X_pca[Y == -1, 1], c='blue', label='Non-Cancerous')  # Non-cancerous in blue
plt.scatter(X_pca[Y == 1, 0], X_pca[Y == 1, 1], c='red', label='Cancerous')  # Cancerous labeled in red

# Adding labels and title
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Image Data')
plt.legend()


In [ ]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.3, random_state=42)

In [ ]:
# Log Regression Model
logreg = LogisticRegression(max_iter=10000)

# Fit the logistic regression model on the training data
logreg.fit(X_train, Y_train)

# Predict the labels for the test set
Y_pred = logreg.predict(X_test)

In [ ]:
# SVM model
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an SVM classifier
clf = svm.SVC(kernel='poly', C=1.0, gamma=0.5, class_weight="balanced")

# Train the classifier
clf.fit(X_train, Y_train)

# Make predictions
Y_pred = clf.predict(X_test)


In [ ]:
conf_matrix = confusion_matrix(Y_test, Y_pred)

# Initialize the ConfusionMatrixDisplay object with the confusion matrix
cmd = ConfusionMatrixDisplay(conf_matrix, display_labels=logreg.classes_)

# Plot the confusion matrix
cmd.plot(cmap=plt.cm.Blues)  # You can choose other color maps like 'viridis', 'plasma', etc.
plt.title('Confusion Matrix')
plt.show()

In [ ]:
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, pos_label=1)
recall = recall_score(Y_test, Y_pred, pos_label=1)
f1 = f1_score(Y_test, Y_pred, pos_label=1)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14,5))

ax1 = axes[0]
ax1.plot(histograms_cancer_avg[0], "r", label="cancer")
ax1.plot(histograms_benign_avg[0], ":r", label="benign")
ax1.set_xlabel("Red Intensity (a.u.)")
ax1.set_ylabel("Counts (a.u.)")
ax1.legend()

ax2 = axes[1]
ax2.plot(histograms_cancer_avg[1], "g", label="cancer")
ax2.plot(histograms_benign_avg[2], ":g", label="benign")
ax2.set_xlabel("Green Intensity (a.u.)")
ax2.set_ylabel("Counts (a.u.)")
ax2.legend()

ax3 = axes[2]
ax3.plot(histograms_cancer_avg[2], "b", label="cancer")
ax3.plot(histograms_benign_avg[2], ":b", label="benign")
ax3.set_xlabel("Blue Intensity (a.u.)")
ax3.set_ylabel("Counts (a.u.)")
ax3.legend()

fig.tight_layout()
fig.savefig("figures/rgb_hist.png", dpi=300, transparent=False)